![Kaggle logo][def]


[def]: https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png

# Import packages

In [1]:
import pandas as pd
import numpy as np 
import plotly.express as px 
import cufflinks as cf
import chart_studio
from  sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier, Pool
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from datetime import date


In [2]:
from utils.names import TARGET
from log_mlflow.log_mlflow import run_mlflow

# Import datasets 

In [3]:
extension = '.csv'
path = 'input/'
name_train = 'train'
name_test = 'test'
name_submisson = 'sample_submission'
path_train = path + name_train + extension
path_test = path + name_test + extension
path_submission = path + name_submisson + extension
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test, delim_whitespace=True)
df_sample_submission = pd.read_csv(path_submission)

In [4]:
df_train.head(10)

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
5,5,703,D-penicillamine,19270,F,N,Y,N,N,0.6,227.0,3.46,34.0,6456.2,60.63,68.0,213.0,11.5,3.0,D
6,6,1300,Placebo,17703,F,N,N,N,N,1.0,328.0,3.35,43.0,1677.0,137.95,90.0,291.0,9.8,3.0,C
7,7,1615,Placebo,21281,F,N,Y,N,N,0.6,273.0,3.94,36.0,598.0,52.70,214.0,227.0,9.9,3.0,C
8,8,2050,D-penicillamine,20684,F,N,N,N,N,0.7,360.0,3.65,72.0,3196.0,94.55,154.0,269.0,9.8,2.0,C
9,9,2615,D-penicillamine,15009,F,N,N,N,N,0.9,478.0,3.60,39.0,1758.0,171.00,140.0,234.0,10.6,2.0,C


In [5]:
df_train.describe()

,id,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000,7905.000000
mean,3952.000000,2030.173308,18373.146490,2.594485,350.561923,3.548323,83.902846,1816.745250,114.604602,115.340164,265.228969,10.629462,3.032511
std,2282.121272,1094.233744,3679.958739,3.812960,195.379344,0.346171,75.899266,1903.750657,48.790945,52.530402,87.465579,0.781735,0.866511
min,0.000000,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,26.350000,33.000000,62.000000,9.000000,1.000000
25%,1976.000000,1230.000000,15574.000000,0.700000,248.000000,3.350000,39.000000,834.000000,75.950000,84.000000,211.000000,10.000000,2.000000
50%,3952.000000,1831.000000,18713.000000,1.100000,298.000000,3.580000,63.000000,1181.000000,108.500000,104.000000,265.000000,10.600000,3.000000
75%,5928.000000,2689.000000,20684.000000,3.000000,390.000000,3.770000,102.000000,1857.000000,137.950000,139.000000,316.000000,11.000000,4.000000
max,7904.000000,4795.000000,28650.000000,28.000000,1775.000000,4.640000,588.000000,13862.400000,457.250000,598.000000,563.000000,18.000000,4.000000


In [6]:
categorial_features = ['id','Drug','Sex','Ascites','Hepatomegaly','Spiders','Edema',TARGET]
numeric_features = [column for column in  df_train.columns if column not in categorial_features]

# Plots (using cutlinks library)

In [7]:
cf.set_config_file(sharing='public',theme='white',offline=True)

colors = ['rgb(255,86,87)','rgb(55,108,138)','rgb(242,217,187)','rgb(99,143,169)']

In [8]:
df_count = df_train.groupby(TARGET).nunique().reset_index()
df_count

,Status,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,C,4965,399,2,367,2,2,2,2,3,86,213,140,153,334,185,148,215,46,4
1,CL,275,134,2,150,2,2,2,2,3,55,107,87,87,134,89,81,95,31,4
2,D,2665,352,2,329,2,2,2,2,3,107,212,146,161,316,186,149,211,42,4


In [9]:
px.bar(df_count, x = TARGET, y = 'id'
       , template = 'xgridoff'
       , color=TARGET
       , title = 'Count of each Status'
       , labels= {'id' : 'Count distinct'}
       , color_discrete_sequence = colors[:3])

In [10]:
numeric_columns = df_train[numeric_features]

correlation_matrix = numeric_columns.corr()

correlation_matrix.iplot(kind='heatmap'
                         , colorscale='spectral'
                         , text=True
                         , title='Correlation Matrix')  # type: ignore

# Split Train / Validation

In [11]:
X = df_train.drop(TARGET,axis = 1)
y = df_train[[TARGET]]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state= 42)
X_train.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
3755,3755,3839,Placebo,18352,M,N,N,N,N,1.2,346.0,3.48,67.0,2184.0,128.65,100.0,269.0,9.9,3.0
4010,4010,3445,Placebo,23445,F,N,Y,Y,N,0.6,286.0,3.94,96.0,856.0,71.30,90.0,410.0,11.5,4.0
2656,2656,1447,Placebo,25873,F,Y,Y,N,N,1.4,151.0,3.40,50.0,559.0,73.53,49.0,234.0,13.0,4.0
4320,4320,71,Placebo,20567,F,Y,Y,Y,N,3.4,260.0,3.01,77.0,728.0,137.95,68.0,190.0,12.0,4.0
1611,1611,1827,Placebo,18964,M,Y,Y,Y,N,1.0,360.0,3.99,380.0,967.0,89.90,154.0,278.0,10.0,3.0


# Predictions using catboost

In [15]:
cat_features = categorial_features #.remove(TARGET)
print(cat_features)

['id', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']


In [16]:
model = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

train_dataset = Pool(data=X_train,
                     label=y_train,
                     cat_features=cat_features)

eval_dataset = Pool(data=X_val,
                    label=y_val,
                    cat_features=cat_features)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')
# Fit model
model.fit(train_dataset,verbose= False)

preds_class = model.predict(eval_dataset)

predictions_proba = model.predict_proba(eval_dataset)


In [17]:
run_mlflow(model,'catboost',y_val,predictions_proba)

In [18]:
logloss = log_loss(y_val, predictions_proba)
print(f"Log Loss on validation Set: {logloss}")

Log Loss on validation Set: 0.5031786665278347


# Predictions using Knn

In [19]:
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)
knn_classifier = KNeighborsClassifier(n_neighbors=10) 
knn_classifier.fit(X_train, y_train)

/Users/samaletisalexis/Library/Caches/pypoetry/virtualenvs/cirrhosis-classification-quvCyCIx-py3.9/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



KNeighborsClassifier(n_neighbors=10)

In [20]:
y_pred_proba = knn_classifier.predict_proba(X_val)

# Compute log loss
loss = log_loss(y_val, y_pred_proba)

print(f"Log Loss on validation Set: {loss}")

Log Loss on validation Set: 1.6354485635727287


/Users/samaletisalexis/Library/Caches/pypoetry/virtualenvs/cirrhosis-classification-quvCyCIx-py3.9/lib/python3.9/site-packages/threadpoolctl.py:1019: RuntimeWarning:

libc not found. The ctypes module in Python 3.9 is maybe too old for this OS.



In [21]:
run_mlflow(knn_classifier,'knn',y_val,y_pred_proba)

# Prediction using Xgboost

In [22]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', n_estimators=100, max_depth=3, learning_rate=0.1,enable_categorical = True)
xgb_classifier.fit(X_train, y_train)
# Make predictions on the test set
y_pred = xgb_classifier.predict(X_val)

y_train = label_encoder.inverse_transform(y_train)
y_val = label_encoder.inverse_transform(y_val)

y_pred_proba = xgb_classifier.predict_proba(X_val)


/Users/samaletisalexis/Library/Caches/pypoetry/virtualenvs/cirrhosis-classification-quvCyCIx-py3.9/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/Users/samaletisalexis/Library/Caches/pypoetry/virtualenvs/cirrhosis-classification-quvCyCIx-py3.9/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [23]:
loss = log_loss(y_val, y_pred_proba)
print(f"Log Loss: {loss}")

Log Loss: 0.46177176903881


In [24]:
run_mlflow(xgb_classifier,'xgboost',y_val,y_pred_proba)

# Using a RF

In [25]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set (probabilities)
y_pred_proba = rf_classifier.predict_proba(X_val)

In [26]:
loss = log_loss(y_val, y_pred_proba)
print(f"Log Loss: {loss}")

Log Loss: 0.5413081453464343


In [27]:
run_mlflow(rf_classifier,'RF',y_val,y_pred_proba)

# Using Light GBM

In [28]:
lgb_classifier = lgb.LGBMClassifier(objective='multiclass', num_class=len(set(y_train)), n_estimators=100, max_depth=3, learning_rate=0.1) # type: ignore
lgb_classifier.fit(X_train, y_train) # type: ignore

# Make predictions on the test set (probabilities)
y_pred_proba = lgb_classifier.predict_proba(X_val)

y_pred_lightxbm = lgb_classifier.predict(X_val)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2183
[LightGBM] [Info] Number of data points in the train set: 5533, number of used features: 26
[LightGBM] [Info] Start training from score -0.452838
[LightGBM] [Info] Start training from score -3.360990
[LightGBM] [Info] Start training from score -1.110247
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

In [29]:
loss = log_loss(y_val, y_pred_proba) # type: ignore
print(f"Log Loss: {loss}")

Log Loss: 0.4560050215358396


In [30]:
run_mlflow(lgb_classifier,'Light GBM',y_val,y_pred_proba)

# Using a stacking model 

In [31]:
# Define base models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(objective='multi:softmax', num_class= 3, n_estimators=100, max_depth=3, learning_rate=0.1)
lgb_model = LGBMClassifier(objective='multiclass', num_class= 3, n_estimators=100, max_depth=3, learning_rate=0.1)

# Define the stacking classifier
stacking_model = StackingClassifier(
    estimators=[('rf', rf_model), ('xgb', xgb_model), ('lgb', lgb_model)], # type: ignore
    final_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
    stack_method='predict_proba',  # Use 'predict_proba' for classification
    cv=5  # Number of cross-validation folds
)

# Train the stacking model
stacking_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = stacking_model.predict(X_val)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001117 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2183
[LightGBM] [Info] Number of data points in the train set: 5533, number of used features: 26
[LightGBM] [Info] Start training from score -0.452838
[LightGBM] [Info] Start training from score -3.360990
[LightGBM] [Info] Start training from score -1.110247
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [32]:
y_pred_proba = stacking_model.predict_proba(X_val)
loss = log_loss(y_val, y_pred_proba)
print(f"Log Loss: {loss}")

Log Loss: 0.6848222999371005


In [33]:
run_mlflow(stacking_model,'Stacking',y_val,y_pred_proba)

# Submit soumission 

In [34]:
df_test.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7905,3839,D-penicillamine,19724,F,N,Y,N,N,1.2,546.0,3.37,65.0,1636.0,151.90,90.0,430.0,10.6,2.0
1,7906,2468,D-penicillamine,14975,F,N,N,N,N,1.1,660.0,4.22,94.0,1257.0,151.90,155.0,227.0,10.0,2.0
2,7907,51,Placebo,13149,F,N,Y,N,Y,2.0,151.0,2.96,46.0,961.0,69.75,101.0,213.0,13.0,4.0
3,7908,2330,D-penicillamine,20510,F,N,N,N,N,0.6,293.0,3.85,40.0,554.0,125.55,56.0,270.0,10.6,2.0
4,7909,1615,D-penicillamine,21904,F,N,Y,N,N,1.4,277.0,2.97,121.0,1110.0,125.00,126.0,221.0,9.8,1.0


In [35]:
X_test = pd.get_dummies(df_test)

In [36]:
prob_predict = pd.DataFrame(lgb_classifier.predict_proba(X_test)).rename(columns= {0 : 'Status_C',1: 'Status_CL',2:'Status_D'}) # type: ignore
prob_predict.head()

,Status_C,Status_CL,Status_D
0,0.542848,0.012516,0.444636
1,0.671361,0.176386,0.152254
2,0.032403,0.007327,0.960270
3,0.947934,0.002673,0.049393
4,0.689025,0.033092,0.277884


In [37]:
df_submission = pd.DataFrame()
df_submission['id'] = X_test['id']
df_submission['Status_C'] = prob_predict['Status_C']
df_submission['Status_CL'] = prob_predict['Status_CL']
df_submission['Status_D'] = prob_predict['Status_D']


In [223]:
name_submissions = 'submissions_'
number_of_today_submission = '_1' 
date_of_sub = str(date.today())
path = 'submissions/'
save_sub = path + name_submissions + date_of_sub + number_of_today_submission + extension
df_submission.to_csv(save_sub,index = False)